## Import des librairies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, auc, roc_curve, confusion_matrix
from xgboost import XGBClassifier

In [2]:
# Commenter l'une ou l'autre lignes suivant que l'on souhaite travailler sur le dataset enrichi ou non. 

data=pd.read_csv('weatherAUS.csv')

# Décommenter cette ligne nous permet de travailler sur le modèle enrichi (Voir enrich.jpnb)
# data=pd.read_csv('enricheredaus_sanscluster.csv')

# A décommenter pour travailler sur le modèle simplifié 
#columns_to_preserve=['Date','Humidity3pm', 'Sunshine', 'varhumidity', 'Pressure9am', 'Rainfall', 'WindGustSpeed','Pressure3pm','RainTomorrow']
#data=data[columns_to_preserve]

#On supprime du Dataframe les lignes où RainTomorrow est nul
liste=data.index[data['RainTomorrow'].isnull()]
data.drop(liste , inplace=True)
data=data.drop(columns='Location',errors='ignore')

In [3]:
data.head(5)

,Date,Humidity3pm,Sunshine,varhumidity,Pressure9am,Rainfall,WindGustSpeed,Pressure3pm,RainTomorrow
0,2008-12-01,22.0,NaN,NaN,1007.7,0.6,44.0,1007.1,No
1,2008-12-02,25.0,NaN,3.0,1010.6,0.0,44.0,1007.8,No
2,2008-12-03,30.0,NaN,5.0,1007.6,0.0,46.0,1008.7,No
3,2008-12-04,16.0,NaN,-14.0,1017.6,0.0,24.0,1012.8,No
4,2008-12-05,33.0,NaN,17.0,1010.8,1.0,41.0,1006.0,No


In [4]:
#On convertit le type de la variable Date
data['Date']=data['Date'].astype('datetime64')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           142193 non-null  datetime64[ns]
 1   Humidity3pm    138583 non-null  float64       
 2   Sunshine       74377 non-null   float64       
 3   varhumidity    137698 non-null  float64       
 4   Pressure9am    128179 non-null  float64       
 5   Rainfall       140787 non-null  float64       
 6   WindGustSpeed  132923 non-null  float64       
 7   Pressure3pm    128212 non-null  float64       
 8   RainTomorrow   142193 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 10.8+ MB


In [6]:
la = LabelEncoder()
data['RainToday'] = la.fit_transform(data['RainToday'])
data['RainTomorrow'] = la.fit_transform(data['RainTomorrow'])

# Data cleaning

In [7]:
data = pd.get_dummies(data)

In [8]:
data = data.dropna()

# Entraînement du modèle

In [9]:
cor = data.corr()
cor2 = data.corr()
cor = np.where(cor <0,cor*-1 , cor)
cor=pd.DataFrame(cor)
cor.columns=cor2.columns
cor.index =cor2.index 

/tmp/ipykernel_394/3685902465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor = data.corr()
/tmp/ipykernel_394/3685902465.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor2 = data.corr()


In [10]:
cor

,Humidity3pm,Sunshine,varhumidity,Pressure9am,Rainfall,WindGustSpeed,Pressure3pm,RainTomorrow
Humidity3pm,1.000000,0.637521,0.418671,0.059048,0.259480,0.052187,0.028476,0.456552
Sunshine,0.637521,1.000000,0.300893,0.045355,0.230080,0.033548,0.014928,0.454024
varhumidity,0.418671,0.300893,1.000000,0.102522,0.060028,0.044272,0.063414,0.267506
Pressure9am,0.059048,0.045355,0.102522,1.000000,0.175320,0.449964,0.961555,0.265992
Rainfall,0.259480,0.230080,0.060028,0.175320,1.000000,0.116457,0.139090,0.237118
WindGustSpeed,0.052187,0.033548,0.044272,0.449964,0.116457,1.000000,0.406659,0.238022
Pressure3pm,0.028476,0.014928,0.063414,0.961555,0.139090,0.406659,1.000000,0.243252
RainTomorrow,0.456552,0.454024,0.267506,0.265992,0.237118,0.238022,0.243252,1.000000


In [11]:
x=data

In [12]:
x = data
y = data['RainTomorrow']
x.drop('RainTomorrow', inplace = True, axis = 1)
x.drop('qtyraintomorrow', inplace = True, axis = 1,errors='ignore')
#x.drop('qtyraintomorrow', inplace = True, axis = 1)
x.drop('Date', inplace = True, axis = 1)
x

,Humidity3pm,Sunshine,varhumidity,Pressure9am,Rainfall,WindGustSpeed,Pressure3pm
6050,8.0,13.0,-5.0,1012.9,0.0,37.0,1012.1
6052,22.0,10.6,15.0,1012.3,0.0,46.0,1009.2
6053,22.0,12.2,0.0,1012.7,0.0,31.0,1009.1
6054,15.0,8.4,-7.0,1010.7,0.0,35.0,1007.4
6055,19.0,0.0,4.0,1007.7,0.0,43.0,1007.4
...,...,...,...,...,...,...,...
142298,32.0,11.0,4.0,1013.9,0.0,35.0,1010.5
142299,28.0,8.6,-4.0,1014.6,0.0,37.0,1011.2
142300,23.0,11.0,-5.0,1015.3,0.0,33.0,1011.8
142301,58.0,10.6,35.0,1014.9,0.0,26.0,1010.7


In [13]:
y.value_counts()

0    53818
1    15214
Name: RainTomorrow, dtype: int64

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

## Interpolation des valeur manquantes

# Evaluation du modèle

In [15]:
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200)

In [16]:
predicted = rf.predict(x_test)
print("La précision est de {} %".format(accuracy_score(predicted, y_test)*100))

La précision est de 86.14470920547548 %


In [17]:
pd.crosstab(predicted, y_test)

RainTomorrow,0,1
row_0,,
0,10199,1344
1,569,1695


In [18]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x, y, cv=5)
print(scores)


[0.84478888 0.85014848 0.84289439 0.87222947 0.85535275]


## Sauvegarde du modèle simplifié (voir enrich.jpnb)

In [21]:
import joblib
filename = 'simplified_model.sav'
#joblib.dump(rf, filename)

['simplified_model.sav']